In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from decorrelation.node_perturbation import NPModule

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [2]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(1),
        transforms.Normalize((0.5), (0.5)),
        torch.flatten # for input to linear layer
        ])

train_data = MNIST(root='~/Data', train=True, download=True, transform=transform)
train_data = Subset(train_data, np.random.permutation(len(train_data.data))[:1000])
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=0, drop_last=True)

In [4]:
# args = argparse.Namespace(lr=0.0, kappa=1e-2, decor_lr=1e-1, bias_lr=0.0, epochs=20)

model = NPModule(nn.Linear(784, 10), device=device)

lossfun = torch.nn.CrossEntropyLoss().to(device)

for batchnum, batch in enumerate(train_loader):
        
    # optimizer.zero_grad() if args.lr > 0.0 else None
    
    input = batch[0].to(device)
    target = batch[1].to(device)

    loss = lossfun(model(input), torch.concat([target, target]))

    # if epoch > 0 and args.lr > 0.0:
    #     optimizer.step()



# res = train(args, model, lossfun, train_loader, device)

